###  

In [4]:
#!conda create -n nlp_tuto python=3.6 anaconda --yes
#!source activate nlp_tuto
#!nvcc --version
!git clone https://github.com/NLP-kr/tensorflow-ml-nlp.git

Cloning into 'tensorflow-ml-nlp'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 842 (delta 30), reused 27 (delta 15), pack-reused 778
Receiving objects: 100% (842/842), 160.14 MiB | 5.74 MiB/s, done.
Resolving deltas: 100% (497/497), done.
Checking out files: 100% (100/100), done.


In [7]:
#tf.data module test
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras import preprocessing
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [20]:
samples = ['너 오늘 이뻐 보인다',
          '나는 오늘 기분이 더러워',
          '끝내주는데, 좋은 일이 있나봐',
          '나 좋은 일이 생겼어',
          '아 오늘 진짜 짜증나',
          '환상적인데, 정말 좋은거 같아']

label = [[1],[0],[1],[1],[0],[1]]

#txt to w_index
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)

word_index = tokenizer.word_index
print(sequences)
print(word_index)

[[4, 1, 5, 6], [7, 1, 8, 9], [10, 2, 3, 11], [12, 2, 3, 13], [14, 1, 15, 16], [17, 18, 19, 20]]
{'정말': 18, '끝내주는데': 10, '짜증나': 16, '기분이': 8, '좋은거': 19, '나는': 7, '이뻐': 5, '오늘': 1, '보인다': 6, '좋은': 2, '너': 4, '있나봐': 11, '진짜': 15, '나': 12, '같아': 20, '아': 14, '생겼어': 13, '환상적인데': 17, '더러워': 9, '일이': 3}


In [16]:
# tf.data
dataset = tf.data.Dataset.from_tensor_slices((sequences, label)) #sequences와 label을 한번에 묶어줌
iterator = dataset.make_one_shot_iterator() #데이터 하나씩 iteration (iter type)
next_data = iterator.get_next() # get data from iter type

with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_data))
        except tf.errors.OutOfRangeError:
            break
#와 짱편하다


(array([4, 1, 5, 6], dtype=int32), array([1], dtype=int32))
(array([7, 1, 8, 9], dtype=int32), array([0], dtype=int32))
(array([10,  2,  3, 11], dtype=int32), array([1], dtype=int32))
(array([12,  2,  3, 13], dtype=int32), array([1], dtype=int32))
(array([14,  1, 15, 16], dtype=int32), array([0], dtype=int32))
(array([17, 18, 19, 20], dtype=int32), array([1], dtype=int32))


In [24]:
#BATCH, shuffle, epoch 적용
BATCH_SIZE = 2
EPOCH = 2
dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
#dataset = dataset.batch(BATCH_SIZE) #batch
#dataset = dataset.shuffle(len(sequences)) #shuffle
dataset = dataset.repeat(len(sequences)) #epoch 
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_data))
        except tf.errors.OutOfRangeError:
            break
            

(array([4, 1, 5, 6], dtype=int32), array([1], dtype=int32))
(array([7, 1, 8, 9], dtype=int32), array([0], dtype=int32))
(array([10,  2,  3, 11], dtype=int32), array([1], dtype=int32))
(array([12,  2,  3, 13], dtype=int32), array([1], dtype=int32))
(array([14,  1, 15, 16], dtype=int32), array([0], dtype=int32))
(array([17, 18, 19, 20], dtype=int32), array([1], dtype=int32))
(array([4, 1, 5, 6], dtype=int32), array([1], dtype=int32))
(array([7, 1, 8, 9], dtype=int32), array([0], dtype=int32))
(array([10,  2,  3, 11], dtype=int32), array([1], dtype=int32))
(array([12,  2,  3, 13], dtype=int32), array([1], dtype=int32))
(array([14,  1, 15, 16], dtype=int32), array([0], dtype=int32))
(array([17, 18, 19, 20], dtype=int32), array([1], dtype=int32))
(array([4, 1, 5, 6], dtype=int32), array([1], dtype=int32))
(array([7, 1, 8, 9], dtype=int32), array([0], dtype=int32))
(array([10,  2,  3, 11], dtype=int32), array([1], dtype=int32))
(array([12,  2,  3, 13], dtype=int32), array([1], dtype=int32))


In [25]:
#mapping - input이 다수일 때 
def mapping_fn(X, Y=None):
    input = {'x':X}
    label = Y
    return input, label

dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.map(mapping_fn) 
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_data))
        except tf.errors.OutOfRangeError:
            break

({'x': array([4, 1, 5, 6], dtype=int32)}, array([1], dtype=int32))
({'x': array([7, 1, 8, 9], dtype=int32)}, array([0], dtype=int32))
({'x': array([10,  2,  3, 11], dtype=int32)}, array([1], dtype=int32))
({'x': array([12,  2,  3, 13], dtype=int32)}, array([1], dtype=int32))
({'x': array([14,  1, 15, 16], dtype=int32)}, array([0], dtype=int32))
({'x': array([17, 18, 19, 20], dtype=int32)}, array([1], dtype=int32))


In [26]:
def mapping_fn(X, Y=None):
    input = {'x':X}
    label = Y
    return input, label

dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.map(mapping_fn) 
dataset = dataset.shuffle(len(sequences)) #shuffle
dataset = dataset.batch(BATCH_SIZE) #batch
dataset = dataset.repeat(len(sequences)) #epoch 
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

with tf.Session() as sess:
    while True:
        try:
            print(sess.run(next_data))
        except tf.errors.OutOfRangeError:
            break

({'x': array([[12,  2,  3, 13],
       [14,  1, 15, 16]], dtype=int32)}, array([[1],
       [0]], dtype=int32))
({'x': array([[ 4,  1,  5,  6],
       [17, 18, 19, 20]], dtype=int32)}, array([[1],
       [1]], dtype=int32))
({'x': array([[10,  2,  3, 11],
       [ 7,  1,  8,  9]], dtype=int32)}, array([[1],
       [0]], dtype=int32))
({'x': array([[ 7,  1,  8,  9],
       [17, 18, 19, 20]], dtype=int32)}, array([[0],
       [1]], dtype=int32))
({'x': array([[10,  2,  3, 11],
       [12,  2,  3, 13]], dtype=int32)}, array([[1],
       [1]], dtype=int32))
({'x': array([[14,  1, 15, 16],
       [ 4,  1,  5,  6]], dtype=int32)}, array([[0],
       [1]], dtype=int32))
({'x': array([[ 7,  1,  8,  9],
       [12,  2,  3, 13]], dtype=int32)}, array([[0],
       [1]], dtype=int32))
({'x': array([[17, 18, 19, 20],
       [10,  2,  3, 11]], dtype=int32)}, array([[1],
       [1]], dtype=int32))
({'x': array([[ 4,  1,  5,  6],
       [14,  1, 15, 16]], dtype=int32)}, array([[1],
       [0]], dtype=